<a href="https://colab.research.google.com/github/alihuss1017/LSTM-Weather-Prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import userdata
gh_token = userdata.get('GITHUB_TOKEN')

In [3]:
!git clone https://{gh_token}@github.com/alihuss1017/LSTM-Weather-Prediction.git

Cloning into 'LSTM-Weather-Prediction'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 9), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 89.45 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [4]:
cd LSTM-Weather-Prediction

/content/LSTM-Weather-Prediction


In [5]:
import pandas as pd
df = pd.read_csv('data/seattle-weather.csv')

## Checking for Null Values and Duplicates

In [6]:
print(f'''Number of null values:\n{df.isnull().sum()}\n\nNumber of duplicated rows: {df.duplicated().sum()}''')

Number of null values:
date             0
precipitation    0
temp_max         0
temp_min         0
wind             0
weather          0
dtype: int64

Number of duplicated rows: 0


## Setting DateTime as Index

In [7]:
df = df.set_index(df["date"])
df = df.drop('date', axis = 1)


## Saving the mean and standard deviation for Inference Purposes

In [8]:
mu, std = df['temp_max'].mean(), df['temp_max'].std()

## One-Hot Encoding Categorical Features:

In [9]:
df_encoded = pd.get_dummies(df, columns = ['weather'])

## Applying Z-Score Normalization on Numerical Features

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
columns_to_normalize = df_encoded.select_dtypes(include='float').columns.tolist()

df_encoded[columns_to_normalize] = scaler.fit_transform(df_encoded[columns_to_normalize])

## Defining the PyTorch Custom Dataset Class

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

class WeatherDataset(Dataset):

  def __init__(self, data_df, seq_len):
    self.data = data_df
    self.seq_len = seq_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    cols_to_cast = self.data.select_dtypes(include = ['object', 'bool']).columns.tolist()
    for col in cols_to_cast:
      self.data[col] = self.data[col].astype('int')

    x = torch.tensor(self.data.iloc[idx:idx+self.seq_len].values, dtype = torch.float32)
    y = torch.tensor(self.data['temp_max'].iloc[idx+self.seq_len+1], dtype = torch.float32)

    return x, y


## Defining Model

In [12]:
import torch.nn as nn

input_features = 9

class lstmModel(nn.Module):
  def __init__(self, hidden_features, num_layers):
    super().__init__()

    self.lstm = nn.LSTM(input_size = input_features,
                        hidden_size = hidden_features, num_layers = num_layers,
                        batch_first = True)
    self.fc = nn.Linear(hidden_features, 1)

  def forward(self, x):
    out, [h, c] = self.lstm(x)
    return self.fc(h[-1])


## Model Debugging

In [13]:
model = lstmModel(32, 2)
model.eval()
with torch.no_grad():
  print(f'Output: {model(torch.rand((5, 9)))}')

Output: tensor([0.0483])


## Configuring Device

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print(f'Device: {device}')

Device: cuda


## Training Model

In [24]:
def train(model, train_loader, optimizer, loss_fn):
  model.train()

  for X, y in train_loader:
    optimizer.zero_grad()

    X, y = X.to(device), y.to(device)
    y_hat = model(X)

    loss = loss_fn(y, y_hat)

    loss.backward()
    optimizer.step()


## Evaluating Model

In [25]:
import matplotlib.pyplot as plt
import numpy as np


def eval(model, val_loader, loss_fn):
  predicted = []
  actual = []
  total_loss = 0
  model.eval()

  with torch.no_grad():
    for X, y in val_loader:
      X, y = X.to(device), y.to(device)
      y_hat = model(X)

      loss = loss_fn(y, y_hat)
      total_loss += loss.item()

  return total_loss / len(val_loader)




In [17]:
!pip install wandb
!pip install optuna

In [18]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alihuss1017 (alihuss1017-uc-san-diego) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [28]:
import optuna
import wandb
from torch.utils.data import Subset
import torch.optim as optim


def objective(trial):

  seq_len = trial.suggest_int('seq_len', 5, 20)
  batch_size = trial.suggest_int('batch_size', 16, 64)
  hidden_features = trial.suggest_int('hidden_features', 32, 128)
  num_layers = trial.suggest_int('num_layers', 1, 3)
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
  num_epochs = trial.suggest_int('num_epochs', 5, 10)
  loss_fn = nn.MSELoss()

  dataset = WeatherDataset(data_df = df_encoded, seq_len = seq_len)
  train_len = int(0.7 * len(dataset))

  train_data = Subset(dataset, range(train_len))
  val_data = Subset(dataset, range(train_len, len(dataset)))

  train_loader = DataLoader(train_data, batch_size = 32, num_workers = 2, drop_last = True)
  val_loader = DataLoader(val_data, batch_size = 32, num_workers = 2, drop_last = True )

  model = lstmModel(hidden_features = hidden_features, num_layers = num_layers).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr = lr)

  for epoch in range(num_epochs):
    train(model, train_loader, optimizer, loss_fn)

  val_loss = eval(model, val_loader, loss_fn)

  return val_loss



In [29]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

best_trial = study.best_trial

[I 2025-11-02 18:26:23,819] A new study created in memory with name: no-name-e47203f6-a512-420c-9aca-3141e463aeb8
/tmp/ipython-input-1740924795.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
[I 2025-11-02 18:26:29,221] Trial 0 finished with value: 0.301921831873747 and parameters: {'seq_len': 18, 'batch_size': 50, 'hidden_features': 77, 'num_layers': 2, 'lr': 0.00012854096279597534, 'num_epochs': 10}. Best is trial 0 with value: 0.301921831873747.
[I 2025-11-02 18:26:33,777] Trial 1 finished with value: 0.3040943317688428 and parameters: {'seq_len': 13, 'batch_size': 19, 'hidden_features': 128, 'num_layers': 3, 'lr': 3.244881579211936e-05, 'num_epochs': 10}. Best is trial 0 with value: 0.301921831873747.
[I 2025-11-02 18:26:37,054] Trial 2 finished with value: 0.300299002574

In [37]:
for key, val in best_trial.params.items():
  print(f"{key}: {val}")

seq_len: 20
batch_size: 28
hidden_features: 91
num_layers: 1
lr: 0.0015898115000807977
num_epochs: 9
